#  #싸이그래머 #레이지앱스 #파트 III #3회차

## 발표내용

### Two Scoops of Django

- 8장 함수 기반 뷰와 클래스 기반 뷰

- 9장 함수 기반 뷰의 모범적인 이용

- 10장 클래스 기반 뷰의 모범적인 이용

---


# Django의 View 란?

##  Django는 MTV(Model-Template-View) 패턴 기반의 웹프레임워크

- MTV 는 MVC(Model-View-Controller)와 유사하다.
- MVC 의 Model 은 MTV 에서도 같은 Model.
- MVC 의 **View** 는 MTV 의 Template.
- MVC 의 Controller 는 MTV 의 **View**.
- 같은 이름의 **View** 라고 하는 것이 서로 다른 역할을 하고 있다.

## MVC 패턴

![MVC패턴](mvc.png)

## Django 의 MTV 패턴

![MTV패턴](mtv.png)

- 사용자가 요청을 하면 URLconf 모듈이 받아서 URL 분석
- URL 분석에 따라 View 를 지정(라우팅)
- View 는 해당 로직을 실행하고, 필요한 경우 Model을 통해 DB 처리를 수행
- 로직에 대한 처리 후 Template 을 통해 HTML 을 생성
- HTML을 사용자에게 응답

---


# Django View 구현

- 함수 기반 뷰(function-based views, FBV) 또는 클래스 기반 뷰(class-based views, CBV) 로 구현할 수 있다.
- 개발자가 편한 방식으로 사용하면 되지만, 하나의 프로젝트에 둘 다 사용하는 경우가 많다.
- Django 에서 제공하는 제너릭 뷰를 사용할 수 있고, 재활용 및 확장성 측면에서도 클래스 기반 뷰를 권장한다.

## 예제

### url.py

In [ ]:
from django.conf.urls import url
from polls import views

# 함수 기반 뷰
urlpatterns = [
    url(r'^$', views.index, name='index'),
    url(r'^(?P<question_id>\d+)/$', views.detail, name='detail'),
    url(r'^(?P<question_id>\d+)/vote/$', views.vote, name='vote'),
    url(r'^(?P<question_id>\d+)/results/$', views.results, name='results'),
]

# 클래스 기반 뷰
urlpatterns = [
    url(r'^$', views.IndexView.as_view(), name='index'),
    url(r'^(?P<pk>\d+)/$', views.DetailView.as_view(), name='detail'),
    url(r'^(?P<question_id>\d+)/vote/$', views.vote, name='vote'),
    url(r'^(?P<pk>\d+)/results/$', views.ResultsView.as_view(), name='results'),
]


### views.py

In [ ]:
from django.shortcuts import get_object_or_404, render
from django.http import HttpResponseRedirect
from django.core.urlresolvers import reverse
from django.views.generic import ListView, DetailView
from polls.models import Choice, Question

import logging
logger = logging.getLogger(__name__)


# 함수 기반 뷰
def index(request):
    latest_question_list = Question.objects.all().order_by('-pub_date')[:5]
    context = {'latest_question_list': latest_question_list}

    return render(request, 'polls/index.html', context)


# 클래스 기반 뷰
class IndexView(ListView):
    template_name = 'polls/index.html'
    context_object_name = 'latest_question_list'

    def get_queryset(self):
        return Question.objects.order_by('-pub_date')[:5]


# 함수 기반 뷰
def detail(request, question_id):
    question = get_object_or_404(Question, pk=question_id)

    return render(request, 'polls/detail.html', {'question': question})


# 클래스 기반 뷰
class DetailView(DetailView):
    model = Question
    template_name = "polls/detail.html"


# 함수 기반 뷰
def results(request, question_id):
    question = get_object_or_404(Question, pk=question_id)

    return render(request, 'polls/results.html', {'question': question})


# 클래스 기반 뷰
class ResultsView(DetailView):
    model = Question
    template_name = "polls/results.html"
    
    
# 함수 기반 뷰
def vote(request, question_id):
    logger.debug("vote().question_id: %s" % question_id)
    p = get_object_or_404(Question, pk=question_id)
    try:
        selected_choice = p.choice_set.get(pk=request.POST['choice'])
    except(KeyError, Choice.DoesNotExist):
        return render(request, 'polls/detail.html', {
            'question': p,
            'error_message': "You didn't select a choice.",
        })
    else:
        selected_choice.votes += 1
        selected_choice.save()

        return HttpResponseRedirect(reverse('polls:results', args=(p.id,)))

# 8장 함수 기반 뷰와 클래스 기반 뷰

## 8.1 함수 기반 뷰와 클래스 기반 뷰를 각각 언제 이용할 것인가?

- 경우에 따라 다르니 매번 뷰를 구현할 때마다 함수 기반 뷰로 해야 할지, 클래스 기반 뷰로 해야 할지를 생각하라.
- 대부분의 경우 클래스 기반 뷰를 선호하며, 특별히 더 복잡해지거나 커스텀 에러 뷰들에 대해서만 함수 기반 뷰를 이용한다.

### 뷰 선택 순서도(그림 8.1)

![순서도](flow.png)

---


## 8.2 URLConf 로부터 뷰 로직을 분리하기

- Django 로 오는 요청들은 urls.py 라는 모듈 내에서 URLConf 를 통해 View 로 라우팅된다.
- Django 의 URL 디자인 철학에 따르면 View 와 URL 의 결합은 최대한의 유연성을 제공하기 위해 느슨하게 구성되어야 한다.
    - View 모듈은 View 로직을 포함해야 한다.
    - URL 모듈은 URL 로직을 포함해야 한다.

### 나쁜 예제 8.1


In [ ]:
from django.conf.urls import url
from django.views.generic import DetailView

from tastings.models import Tasting

urlpatterns = [
    url(r"^(?P<pk>\d+)/$",
       DetailView.as_view(
            model=Tasting,
            template_name="tastings/detail.html"),
       name="detail"),
    url(r"^(?P<pk>\d+)/results/$",
       DetailView.as_view(
            model=Tasting,
            template_name="tastings/results.html"),
       name="results"),
]


### 위 코드는 Django 의 디자인 철학에 어긋난다.

- View, URL, Model 사이에 느슨한 결합(loose coupling)이 안되고, 종속적인 결합(tight coupling)이 되어 있다.
- 같거나 비슷한 인자를 반복해서 이용하고 있다.


## URLConf 에서 느슨한 결합 유지하기

### 예제 8.1, 8.2


In [ ]:
# tastings/views.py
from django.views.generic import ListView, DetailView, UpdateView
from django.core.urlresolvers import reverse

from .models import Tasting

class TasteListView(ListView):
    model = Tasting

class TasteDetailView(DetailView):
    model = Tasting

class TasteResultsView(TasteDetailView):
    template_name = "tastings/results.html"

class TasteUpdateView(UpdateView):
    model = Tasting

    def get_success_url(self):
        return reverse("tastings:detail",
            kwargs={"pk": self.object.pk})
    

# tastings/urls.py
from django.conf.urls import url

from . import views

urlpatterns = [
    url(
        regex=r"^$",
        view=views.TasteListView.as_view(),
        name="list"
    ),
    url(
        regex=r"^(?P<pk>\d+)/$",
        view=views.TasteDetailView.as_view(),
        name="detail"
    ),
    url(
        regex=r"^(?P<pk>\d+)/results/$",
        view=views.TasteResultsView.as_view(),
        name="results"
    ),
    url(
        regex=r"^(?P<pk>\d+)/update/$",
        view=views.TasteUpdateView.as_view(),
        name="update"
    )
]



## 8.4 URL 이름공간 이용하기

- 8.4.1 URL 이름을 짧고, 명확하고, 반복되는 작업을 피해서 작성하는 방법
- 8.4.2 서드 파티 라이브러리와 상호 운영성을 높이기
- 8.4.3 검색, 업그레이드, 리팩터링을 쉽게 하기
- 8.4.4 더 많은 앱과 템플릿 리버스 트릭을 허용하기

### 예제 8.3, 8.4, 8.5


In [ ]:
# urls.py at root of project
urlpatterns += [
    url(r'^tastings/', include('tastings.urls', namespace='tastings')),
]


# tastings/views.py snippet
class TasteUpdateView(UpdateView):
    model = Tasting

    def get_success_url(self):
        return reverse("tastings:detail", 
            kwargs={"pk": self.object.pk})
    

# HTML 템플릿
{% extends "base.html" %}

{% block title %}Tastings{% endblock title %}

{% block content %}
<ul>
  {% for taste in tastings %}
    <li>
      <a href="{% url "tastings:detail" taste.pk %}">{{ taste.title }}</a>
      <small>
        (<a href="{% url "tastings:update" taste.pk %}">update</a>)
      </small>
    </li>
  {% endfor %}
</ul>
{% endblock content %}



## URLConf 에서 뷰를 문자열로 지목하지 말자

- Django 1.8 이전의 튜토리얼에서는 View 를 지정하는데 문자열을 썼다.

### 나쁜 예제 8.2, 예제 8.8


In [ ]:
# 절대 따라하지 말자!
# polls/urls.py
from djago.conf.urls import patterns, url

urlpatterns = patterns('',
    # 뷰를 문자열로 정의
    url(r'^$', 'polls.views.index', name='index'),
)


# polls/urls.py
from django.conf.urls import url

from . import views

urlpatterns = [
    # Defining the views explicitly
    url(r'^$', views.index, name='index'),
]



## 8.6 뷰에서 비즈니스 로직 분리하기

- 표준적으로 이용되는 구조 이외에 덧붙여진 비즈니스 로직을 발견할 때마다 View 밖으로 이동시키자.
- 모델 메서드, 매니저 메서도 또는 일반적인 유틸리티 헬퍼 함수들을 이용하자.
- 비즈니스 로직이 쉽게 재사용 가능한 컴포넌트가 되고 이를 View 에서 호출하는 경우 확장하기가 쉬워진다.


## 8.7 장고의 뷰와 함수

- 클래스 기반 뷰의 경우 실제로는 함수로 호출된다.
- URLConf 에서 View.as_view() 라는 클래스 메서드는 실제로 호출 가능한 뷰 인스턴스를 반환한다.
- 즉, 콜백 함수 자체가 함수 기반 뷰와 동일하게 작동한다.

### 8.7.1 뷰의 기본 형태들

- Django 의 함수 기반 뷰는 HTTP 메서드에 중립적이지만, 클래스 기반 뷰의 경우 HTTP 메서드의 선언이 필요하다.

### 예제 8.10


In [ ]:
# simplest_views.py
from django.http import HttpResponse
from django.views.generic import View

# The simplest FBV
def simplest_view(request):
    # Business logic goes here
    return HttpResponse("FBV")

# The simplest CBV
class SimplestView(View):
    def get(self, request, *args, **kwargs):
        # Business logic goes here
        return HttpResponse("CBV")


# 9장 함수 기반 뷰의 모범적인 이용

## 9.1 함수 기반 뷰의 장점

- 함수 기반 뷰의 단순함은 사실 코드 재사용을 희생하여 나온 결과다.
- 함수 기반 뷰로 코드 작성 시 가이드라인
    - View 코드는 작을수록 좋다.
    - View 에서 절대 코드를 반복해서 사용하지 말자.
    - View 는 프레젠테이션 로직을 처리해야 한다. 비즈니스 로직은 가능한 모델 로직에 적용시키고 만약 해야 한다면 폼 안에 내재시켜야 한다.
    - View 를 가능한 단순하게 유지하자.
    - 403, 404, 500 을 처리하는 커스텀 코드를 쓰는데 이용하라.
    - 복잡하게 중첩된 if 블록 구문을 피하자.

## 9.2 HttpRequest 객체 전달하기

- 프로젝트 전체를 아우르는 유틸리티 함수를 만드는 것을 추천한다.
- HttpRequest 객체를 주된 인자(primary argument)로 전달
    - 메서드의 인자 구성을 단순하게 해준다.
    - 함수, 메서드의 인자를 관리하는 데 있어서 부하가 적다.
    
### 예제 9.1, 9.2, 9.3, 9.4


In [ ]:
# sprinkles/utils.py
from django.core.exceptions import PermissionDenied

def check_sprinkle_rights(request):
    if request.user.can_sprinkle or request.user.is_staff:
        return request

    # Return a HTTP 403 back to the user
    raise PermissionDenied 
    
    
# sprinkles/utils.py
from django.core.exceptions import PermissionDenied

def check_sprinkles(request):
    if request.user.can_sprinkle or request.user.is_staff:
        # By adding this value here it means our display templates
        #   can be more generic. We don't need to have
        #   {% if request.user.can_sprinkle or request.user.is_staff %}
        #   instead just using
        #   {% if request.can_sprinkle %}
        request.can_sprinkle = True
        return request

    # Return a HTTP 403 back to the user
    raise PermissionDenied
    
    
# sprinkles/views.py
from django.shortcuts import get_object_or_404
from django.shortcuts import render

from .utils import check_sprinkles
from .models import Sprinkle

def sprinkle_list(request):
    """Standard list view"""

    request = check_sprinkles(request)

    return render(request,
        "sprinkles/sprinkle_list.html",
        {"sprinkles": Sprinkle.objects.all()})

def sprinkle_detail(request, pk):
    """Standard detail view"""

    request = check_sprinkles(request)

    sprinkle = get_object_or_404(Sprinkle, pk=pk)

    return render(request, "sprinkles/sprinkle_detail.html",
        {"sprinkle": sprinkle})

def sprinkle_preview(request):
    """"preview of new sprinkle, but without the
            check_sprinkles function being used.
    """
    sprinkle = Sprinkle.objects.all()
    return render(request,
        "sprinkles/sprinkle_preview.html",
        {"sprinkle": sprinkle})


# sprinkles/views.py
from django.views.generic import DetailView

from .utils import check_sprinkles
from .models import Sprinkle

class SprinkleDetail(DetailView):
    """Standard detail view"""

    model = Sprinkle

    def dispatch(self, request, *args, **kwargs):
        request = check_sprinkles(request)
        return super(SprinkleDetail, self).dispatch(
                                request, *args, **kwargs)
    


## 편리한 데코레이터

- 파이썬에서 데코레이터는 꼭 필요해서가 아니라 코드를 좀 더 간결하게 해주고, 사람이 읽기 편하게 해주는 기능을 하고 있다.
- 함수가 주는 단순 명료함에 데코레이터의 간편표기법을 같이 사용하면 매우 유용하고 강력하다.

### 예제 9.5, 9.6, 9.7


In [ ]:
# simple decorator template
import functools

def decorator(view_func):
    @functools.wraps(view_func)
    def new_view_func(request, *args, **kwargs):
        # You can modify the request (HttpRequest) object here.
        response = view_func(request, *args, **kwargs)
        # You can modify the response (HttpResponse) object here.
        return response
    return new_view_func


# sprinkles/decorators.py
from functools import wraps

from . import utils

# based off the decorator template from Example 8.5
def check_sprinkles(view_func):
    """Check if a user can add sprinkles"""
    @wraps(view_func)
    def new_view_func(request, *args, **kwargs):
        # Act on the request object with utils.can_sprinkle()
        request = utils.can_sprinkle(request)

        # Call the view function
        response = view_func(request, *args, **kwargs)

        # Return the HttpResponse object
        return response
    return new_view_func


# views.py
from django.shortcuts import get_object_or_404, render

from .decorators import check_sprinkles
from .models import Sprinkle

# Attach the decorator to the view
@check_sprinkles
def sprinkle_detail(request, pk):
    """Standard detail view"""

    sprinkle = get_object_or_404(Sprinkle, pk=pk)

    return render(request, "sprinkles/sprinkle_detail.html",
        {"sprinkle": sprinkle})



# 10장 클래스 기반 뷰의 모범적인 이용

- 클래스 기반 뷰에서는 View 클래스가 내장함수를 반환하는 as_view() 클래스 메서드를 제공한다.
- django.views.generic.View 에서 해당 메커니즘이 구현되며, 모든 클래스 기반 뷰는 이 클래스를 직간접적으로 상속받는다.

## 10.1 클래스 기반 뷰를 이용할 때의 가이드라인

- 뷰 코드의 양은 적으면 적을수록 좋다.
- 뷰 안에서 같은 코드를 반복적으로 이용하지 말자.
- 뷰는 프레젠테이션 로직에서 관리하도록 하자. 비즈니스 로직은 모델에서 처리하자. 매우 특별한 경우에는 폼에서 처리하자.
- 뷰는 간단 명료해야 한다.
- 403, 404, 500 에러 핸들링에 클래스 기반 뷰는 이용하지 않는다. 대신 함수 기반 뷰를 이용하자.
- 믹스인은 간단 명료해야 한다.

## 10.2 클래스 기반 뷰와 믹스인 이용하기

- 믹스인은 실체화된 클래스가 아니라 상속해 줄 기능들을 제공하는 클래스를 의미한다.
- 다중 상속을 해야 할 때 믹스인을 쓰면 클래스에 더 나은 기능과 역할을 제공할 수 있다.
- 믹스인을 이용한 View 클래스 구현 시 상속에 관한 규칙(왼쪽에서 오른쪽의 순서로 처리)
    1. Django 가 제공하는 기본 View 는 '항상' 오른쪽으로 진행한다.
    2. 믹스인은 기본 View 에서부터 왼쪽으로 진행한다.
    3. 믹스인은 파이썬의 기본 객체 타입을 상속해야만 한다.

### 예제 10.1


In [ ]:
from django.views.generic import TemplateView

class FreshFruitMixin(object):

    def get_context_data(self, **kwargs):
        context = super(FreshFruitMixin,
                    self).get_context_data(**kwargs)
        context["has_fresh_fruit"] = True
        return context

class FruityFlavorView(FreshFruitMixin, TemplateView):
    template_name = "fruity_flavor.html"
    


## 10.3 어떤 장고 제네릭 클래스 기반 뷰를 어떤 태스크에 이용할 것인가?

### 표 10.1 장고 클래스 기반 뷰의 이용 표

![View](view.png)


## 10.4 장고 클래스 기반 뷰에 대한 일반적인 팁

### 10.4.1 인증된 사용자에게만 장고 클래스 기반 뷰/제네릭 클래스 기반 뷰 접근 가능하게 하기

### 예제 10.2

In [ ]:
# flavors/views.py
from django.views.generic import DetailView

from braces.views import LoginRequiredMixin

from .models import Flavor

class FlavorDetailView(LoginRequiredMixin, DetailView):
    model = Flavor
    


### 10.4.2  뷰에서 유효한 폼을 이용하여 커스텀 액션 구현하기

### 예제 10.3

In [ ]:
from django.views.generic import CreateView

from braces.views import LoginRequiredMixin

from .models import Flavor

class FlavorCreateView(LoginRequiredMixin, CreateView):
    model = Flavor
    fields = ('title', 'slug', 'scoops_remaining')

    def form_valid(self, form):
        # Do custom logic here
        return super(FlavorCreateView, self).form_valid(form)
    


### 10.4.3 뷰에서 부적합한 폼을 이용하여 커스텀 액션 구현하기

### 예제 10.4

In [ ]:
from django.views.generic import CreateView

from braces.views import LoginRequiredMixin

from .models import Flavor

class FlavorCreateView(LoginRequiredMixin, CreateView):
    model = Flavor

    def form_invalid(self, form):
        # Do custom logic here
        return super(FlavorCreateView, self).form_invalid(form)
    


### 10.4.4 뷰 객체 이용하기

### 예제 10.5, 10.6

In [ ]:
from django.utils.functional import cached_property
from django.views.generic import UpdateView, TemplateView

from braces.views import LoginRequiredMixin

from .models import Flavor
from .tasks import update_users_who_favorited

class FavoriteMixin(object):

    @cached_property
    def likes_and_favorites(self):
        """Returns a dictionary of likes and favorites"""
        likes = self.object.likes()
        favorites = self.object.favorites()
        return {
            "likes": likes,
            "favorites": favorites,
            "favorites_count": favorites.count(),

        }

class FlavorUpdateView(LoginRequiredMixin, FavoriteMixin, UpdateView):
    model = Flavor
    fields = ('title', 'slug', 'scoops_remaining')

    def form_valid(self, form):
        update_users_who_favorited(
            instance=self.object,
            favorites=self.likes_and_favorites['favorites']
        )
        return super(FlavorCreateView, self).form_valid(form)

class FlavorDetailView(LoginRequiredMixin, FavoriteMixin, TemplateView):
    model = Flavor
    
    
{# flavors/base.html #}
{% extends "base.html" %}

{% block likes_and_favorites %}
<ul>
  <li>Likes: {{ view.likes_and_favorites.likes }}</li>
  <li>Favorites: {{ view.likes_and_favorites.favorites_count }}</li>
</ul>
{% endblock likes_and_favorites %}
    


## 10.5 제네릭 클래스 기반 뷰와 폼 사용하기

### 10.5.1 뷰 + 모델폼 예제

- 가장 단순하고 일반적인 Django 폼 시나리오
- Model 을 생성한 후 새로운 레코드를 추가하거나 수정하는 기능
- Django 의 작명 관례(naming convention)에 따라 FlavorCreateView, FlavorUpdateView, FlavorDetailView

### 예제 10.8


In [ ]:
# flavors/views.py
from django.views.generic import CreateView, UpdateView, DetailView

from braces.views import LoginRequiredMixin

from .models import Flavor

class FlavorCreateView(LoginRequiredMixin, CreateView):
    model = Flavor
    fields = ('title', 'slug', 'scoops_remaining')

class FlavorUpdateView(LoginRequiredMixin, UpdateView):
    model = Flavor
    fields = ('title', 'slug', 'scoops_remaining')

class FlavorDetailView(DetailView):
    model = Flavor
    


- 확인 페이지를 수정하기 위해 form_valid() 를 오버라이딩하는 믹스인을 만들어 여러 뷰에서 상속한다.

### 예제 10.9, 10.10


In [ ]:
# flavors/views.py
from django.contrib import messages
from django.views.generic import CreateView, UpdateView, DetailView

from braces.views import LoginRequiredMixin

from .models import Flavor

class FlavorActionMixin(object):

    fields = ('title', 'slug', 'scoops_remaining')

    @property
    def success_msg(self):
        return NotImplemented

    def form_valid(self, form):
        messages.info(self.request, self.success_msg)
        return super(FlavorActionMixin, self).form_valid(form)

class FlavorCreateView(LoginRequiredMixin, FlavorActionMixin,
                        CreateView):
    model = Flavor
    success_msg = "Flavor created!"

class FlavorUpdateView(LoginRequiredMixin, FlavorActionMixin,
                        UpdateView):
    model = Flavor
    success_msg = "Flavor updated!"

class FlavorDetailView(DetailView):
    model = Flavor
    
    
{# templates/flavors/flavor_detail.html #}
{% if messages %}
    <ul class="messages">
        {% for message in messages %}
        <li id="message_{{ forloop.counter }}"
            {% if message.tags %} class="{{ message.tags }}"
                {% endif %}>
            {{ message }}
        </li>
        {% endfor %}
    </ul>
{% endif %}



### 10.5.2 뷰 + 폼 예제

- 검색 폼과 같은 경우 ModelForm 이 아니라 장고 Form 을 주로 이용한다.

### 예제 10.11, 10.12


In [ ]:
from django.views.generic import ListView

from .models import Flavor

class FlavorListView(ListView):
    model = Flavor

    def get_queryset(self):
        # Fetch the queryset from the parent get_queryset
        queryset = super(FlavorListView, self).get_queryset()

        # Get the q GET parameter
        q = self.request.GET.get("q")
        if q:
            # Return a filtered queryset
            return queryset.filter(title__icontains=q)
        # Return the base queryset
        return queryset

    
{# templates/flavors/_flavor_search.html #}
{% comment %}
    Usage: {% include "flavors/_flavor_search.html" %}
{% endcomment %}
<form action="{% url "flavor_list" %}" method="GET">
    <input type="text" name="q" />
    <button type="submit">search</button>
</form>
    


## 10.6 django.views.generic.View 이용하기

- 모든 View 에서 django.views.generic.View 만 이용하여 프로젝트 전부를 구성할 수도 있다.

### 예제 10.13


In [ ]:
from django.shortcuts import get_object_or_404
from django.shortcuts import render, redirect
from django.views.generic import View

from braces.views import LoginRequiredMixin

from .forms import FlavorForm
from .models import Flavor

class FlavorView(LoginRequiredMixin, View):

    def get(self, request, *args, **kwargs):
        # Handles display of the Flavor object
        flavor = get_object_or_404(Flavor, slug=kwargs['slug'])
        return render(request,
            "flavors/flavor_detail.html",
                {"flavor": flavor}
            )

    def post(self, request, *args, **kwargs):
        # Handles updates of the Flavor object
        flavor = get_object_or_404(Flavor, slug=kwargs['slug'])
        form = FlavorForm(request.POST)
        if form.is_valid():
            form.save()
        return redirect("flavors:detail", flavor.slug)
    


- GET 메서드와 함께 json, PDF 또는 다른 비 HTML 콘텐츠를 서비스하기 위해서는 django.views.generic.View 클래스가 매우 유용하다.

### 예제 10.14


In [ ]:
from django.http import HttpResponse
from django.shortcuts import get_object_or_404
from django.views.generic import View

from braces.views import LoginRequiredMixin

from .models import Flavor
from .reports import make_flavor_pdf

class PDFFlavorView(LoginRequiredMixin, View):

    def get(self, request, *args, **kwargs):
        # Get the flavor
        flavor = get_object_or_404(Flavor, slug=kwargs['slug'])

        # create the response
        response = HttpResponse(content_type='application/pdf')

        # generate the PDF stream and attach to the response
        response = make_flavor_pdf(response, flavor)

        return response
    


- **객체지향의 장점을 살린 클래스 기반 뷰와 함수 기반 뷰를 서로 조합해서 이용함으로써 그 장점을 최대한 살릴 수 있다.**